In [ ]:
# to allow importing from the parent directory
import sys

# setting path
sys.path.append('../')

## Autoencoder

In [ ]:
from configs.hierarchical_ordered import Config
from utils import load_pretrained_autoencoder
import torch 

cfg = Config()

cfg.data_dir = "../dataset/icons_tensor/"
cfg.meta_filepath = "../dataset/icons_meta.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"

vae_model = load_pretrained_autoencoder(cfg, pretrained_path='../pretrained/hierarchical_ordered.pth.tar', device=device)

## Training

In [6]:
import wandb
import torch
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from dataset.dataset import num_classes, dataloader_with_transformed_dataset
from utils import create_model, train_with_wandb

def sweep_pipeline():
    config_defaults = {
        'optimizer': 'adam',
        'predict_xstart': True,
        'learn_sigma': True,
        'use_scheduler': True,
        'num_heads': 16,
        'depth': 28,
        'dropout': 0.1,
        'epochs': 100,
        'learning_rate': 0.001,
        'batch_size': 100,
        'magical_number': 0.7128, # mean of std's of latents
    }
    wandb.init(config=config_defaults)
    config = wandb.config
    
    train_dataloader, valid_dataloader = dataloader_with_transformed_dataset(vae_model, cfg, batch_n=config.batch_size, length=1000)

    model, diffusion = create_model(dropout=config.dropout, predict_xstart=config.predict_xstart,
                                    n_classes=num_classes(train_dataloader), depth=config.depth, 
                                    learn_sigma=config.learn_sigma, num_heads=config.num_heads)
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=0)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5)
    
    train_with_wandb(model, train_dataloader, diffusion, optimizer, scheduler, config, device=device)

A single training run for debugging.

In [ ]:
sweep_pipeline()
wandb.finish()

# Defining the Sweep

## Config

In [ ]:
import wandb

wandb.login()

In [8]:
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'optimizer': {
            'values': ['adam']
        },
        'predict_xstart': {
            'value': True,
        },
        'learn_sigma':{
            'value': True, # [True, False]
        },
        'use_scheduler':{
            'values': [True, False],
        },
        'num_heads': {
            'values': [16, 32, 64, 128, 256]
        },
        'depth': {
            'distribution': 'int_uniform',
            'min': 28,
            'max': 100, 
        },
        'dropout': {
              'values': [0.3, 0.4, 0.5]
        },
        'epochs': {
            'value': 100
        },
        'learning_rate': {
            # a flat distribution between 0.01 and 0.0001
            'distribution': 'uniform',
            'min': 0.0001,
            'max': 0.01
        },
        'batch_size': {
            # integers between 32 and 256
            # with evenly-distributed logarithms 
            'distribution': 'q_log_uniform_values',
            'q': 8,
            'min': 16,
            'max': 128,
        }
    },
}

## Sweep

In [ ]:
import wandb

sweep_id = wandb.sweep(sweep_config, project="svgfusion-sweep")

In [ ]:
import wandb

wandb.agent(sweep_id, sweep_pipeline, count=100)